## Bibliotecas

In [20]:
import os
import cv2
import numpy as np
from PIL import Image

## Funções

In [27]:
# Binarização dos blocos via método ExG
def binarize_image_exg(img):
    
    B, G, R = cv2.split(img)
    # Fórmula do ExG
    exg = 2 * G.astype(np.float32) - R.astype(np.float32) - B.astype(np.float32)
    # Normalização
    exg_normalized = cv2.normalize(exg, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # Binariza a img através do threshold
    _, binary_img = cv2.threshold(exg_normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return binary_img

## Leitura da imagem e divisão em blocos

In [24]:

# Caminhos de input e output
input_path = r"C:\Users\mathe\Desktop\BemAgro\dados\Orthomosaico_roi.tif"
output_path = r"C:\Users\mathe\Desktop\BemAgro\dados\blocos"
os.makedirs(output_path, exist_ok=True)

In [3]:
# Abrindo a imagem
image = Image.open(input_path)

In [5]:
# Tamanho dos blocos em pixels e coleta da largura a altura da imagem
block_size = 128
width, height = image.size

# Divisão da imagem em blocos
for x in range(0, width, block_size):
    for y in range(0, height, block_size):

        left = x
        upper = y
        right = min(x + block_size, width)
        lower = min(y + block_size, height)
        
        block = image.crop((left, upper, right, lower))

        block_filename = os.path.join(output_path, f"block_{x}_{y}.png")
        block.save(block_filename)

## Geração do Dataset

In [11]:
# Retira-se os blocos que são "rebarbas" dos cortes
min_block_size = 50 * 1024 # 50KB

for filename in os.listdir(output_path):

    path = output_path + '\\' + filename

    block_file_size = os.path.getsize(path)

    if (block_file_size > min_block_size):
        pass
    else:
        os.remove(path)


In [25]:
# Atualizando os diretórios
input_path = output_path
output_path = r"C:\Users\mathe\Desktop\BemAgro\dados\blocos_binarizados"
os.makedirs(output_path, exist_ok=True)

In [22]:
# Aplica a binarização
for filename in os.listdir(input_path):

    input_img_path = input_path + '\\' + filename
    img = cv2.imread(input_img_path)
    
    binary_img = binarize_image_exg(img)
    
    output_img_path = output_path + '\\' + filename
    cv2.imwrite(output_img_path, binary_img)